In [1]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())
adj1 = np.zeros_like(adj1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 9


100%|██████████| 320/320 [00:00<00:00, 406.34it/s]


(0.6279311431977952,
 0.35402500000000003,
 0.4556375,
 0.33663945218091595,
 0.380307566673866)

In [3]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {'average': 0, 'direct': 0, 'indirect': 0, 'spillover_effect':0}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']

ground_truth1

100%|██████████| 200/200 [00:00<00:00, 590.40it/s]


{'average': 0.5736500000000001,
 'direct': 0.3163625000000001,
 'indirect': 0.0,
 'spillover_effect': 0.0}

In [4]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.7
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_raw at 0x12b254c10> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [5]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_raw at 0x12b254c10> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [6]:
# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

print(compute_stats(results1, results2, ground_truth1, adj1))

100%|██████████| 100/100 [00:01<00:00, 99.82it/s]

{'coverage_rate': array([0.92, 0.91, 0.72, 0.42]), 'bias': array([8.48261234e-03, 8.53087996e-03, 1.73472348e-19, 5.69680479e-19]), 'mse': array([1.14273185e-03, 2.51429162e-03, 6.07443407e-04, 4.83516994e-34]), 'var': array([1.07077713e-03, 2.44151570e-03, 6.07443407e-04, 4.83192458e-34]), 'ci_length': array([1.22933233e-01, 1.84717815e-01, 9.54668975e-02, 2.40504966e-17]), 'true_effect': array([0.57365  , 0.3163625, 0.       , 0.       ])}


In [12]:
from tqdm import tqdm
from drnet import compute_avg_effects_std_from_raw

results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)

var = []
var_sim = []
est = []
idx = 3
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(results1[i,:,idx],adj1,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(results1[i,:,idx]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

100%|██████████| 100/100 [00:00<00:00, 407.80it/s]

0.44950518749922275 0.44950518749922297 0.4015949505765826
